# *RUN THIS CHUNK*

In [15]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# First/reset calculations?
#### Set up results table

In [27]:
# make table to hold vectors
full_table = pd.DataFrame({
    'element' : [],
    'instrument' : [],
    'atmosphere' : [],
    'num_range' : [],
    'vector' : [],
    'LOB_Braga' : [],
    'LOB_metals' : [],
    'LOD_Braga' : [],
    'LOD_metals' : [],
    'LOQ_Braga' : [],
    'LOQ_metals' : []
})

# Adding new data?
#### Import existing table

In [16]:
fpath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LOD_calcs.csv"
full_table = pd.read_csv(fpath)

# \*\*\*Start here with a new calculation\*\*\*
## Calculate regression vectors
Data should be in a `XX_coeffs.csv` format within folders separated by random number region (0-750 or 250-1000)

- **LANL 0-750:** C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LANL calculations\\models\\0-750\\
- **LANL 250-1000:** C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LANL calculations\\models\\250-1000\\

#### Get data information

In [36]:
instrument = input("What instrument was the data collected on? ")
atm = input("What atmosphere was the data collected under? ")
num_range = input("What range of random numbers were the models trained on? (0-750 or 250-1000)")
folder = input('Folder path containing models:')

coefflist = os.listdir(folder)
coeffs = []

What instrument was the data collected on? LANL
What atmosphere was the data collected under? Mars
What range of random numbers were the models trained on? (0-750 or 250-1000)250-1000
Folder path containing models:C:\Users\ytsma22c\Google Drive\Misc Work\LOD paper\LANL calculations\models\250-1000\


#### Read in model coefficients

In [37]:
print("Reading in models...")
for file in tqdm(coefflist):
    if "_coeff" in file:
        #get path name        
        path = (folder + file)
        #read in file
        data = pd.read_csv(path, skiprows = [0])
        #merge together data
        coeffs.append(data)

# convert to dataframe
coeffs = pd.concat(coeffs).T
coeffs.columns = coeffs.iloc[0]
coeffs = coeffs.drop(coeffs.index[0])

Reading in models...


  0%|          | 0/37 [00:00<?, ?it/s]

#### Calculate regression vectors

In [38]:
#square root of sum of squares
vectors = coeffs.pow(2).sum().pow(.5)

In [39]:
vectors

wavelength
Li (ppm)       22791.605649
MnO (wt%)          6.508857
Na2O (wt%)       139.356633
Ni (ppm)        6555.697404
Pb (ppm)         932.453539
Rb (ppm)        4578.040971
SiO2 (wt%)      6311.901011
Sr (ppm)      155661.441953
Zn (ppm)        17189.66734
dtype: object

#### Assemble table

In [40]:
# add data to table
vals = vectors
elem = coeffs.columns
instr = [instrument] * len(vectors)
atmos = [atm] * len(vectors)
num = [num_range] * len(vectors)

temp = pd.DataFrame({'element' : elem,
                     'instrument' : instr,
                     'atmosphere' : atmos,
                     'num_range' : num,
                     'vector' : vals
}).reset_index(drop = True)

## Calculate LOB, LOD, LOQ

- LOB = 1.645 * stdev * vector
- LOD = 3.3 * stdev * vector
- LOQ = 10 * stdev * vector

Currently just using the pre-calculated `standard deviation` but can easily make a code for that too. The files just aren't in the best format for LANL right now

#### Assign standard deviation based on instrument and atmosphere

In [41]:
if instrument == 'LANL':
    braga =  0.000245701982436336
    metals = 0.0000919578459982072
    
if instrument == 'ChemLIBS':
    if atm == 'Mars':
        braga = 0.00028197645372017
        metals = 0.000117639483634021
        
    elif atm == 'Earth':
        braga = 0.000444867993101603
        metals = 0.0000931716553399781
    
    else:
        braga = 0.000417553099169204
        metals = 0.000231951753003813

In [42]:
temp['LOB_Braga'] = 1.645 * braga * temp['vector']
temp['LOB_metals'] = 1.645 * metals * temp['vector']
temp['LOD_Braga'] = 3.3 * braga * temp['vector']
temp['LOD_metals'] = 3.3 * metals * temp['vector']
temp['LOQ_Braga'] = 10 * braga * temp['vector']
temp['LOQ_metals'] = 10 * metals * temp['vector']

### Add data to full table

In [43]:
full_table = full_table.append(temp).reset_index(drop=True)
# drop unit info
full_table.element = full_table.apply(lambda row: row.element.split()[0], axis = 1)
display(full_table)

,element,instrument,atmosphere,num_range,vector,LOB_Braga,LOB_metals,LOD_Braga,LOD_metals,LOQ_Braga,LOQ_metals
0,Li,LANL,Mars,0-750,22611.808712,9.139235,3.420503,18.334029,6.8618,55.557662,20.793332
1,MnO,LANL,Mars,0-750,5.756295,0.002327,0.000871,0.004667,0.001747,0.014143,0.005293
2,Na2O,LANL,Mars,0-750,127.19417,0.051409,0.019241,0.103131,0.038598,0.312519,0.116965
3,Ni,LANL,Mars,0-750,9851.998094,3.981978,1.490318,7.988163,2.989696,24.206555,9.059685
4,Pb,LANL,Mars,0-750,544.969803,0.220266,0.082438,0.441871,0.165377,1.339002,0.501142
5,Rb,LANL,Mars,0-750,10578.843708,4.275755,1.600269,8.577501,3.210265,25.992429,9.728077
6,SiO2,LANL,Mars,0-750,7888.620015,3.188421,1.193317,6.396224,2.393888,19.382496,7.254205
7,Sr,LANL,Mars,0-750,130391.36114,52.701549,19.724387,105.723473,39.568679,320.374159,119.905087
8,Zn,LANL,Mars,0-750,9070.461655,3.666097,1.372095,7.35448,2.75253,22.286304,8.341001
9,Li,LANL,Mars,250-1000,22791.605649,9.211906,3.447701,18.479811,6.916361,55.999427,20.95867


### Export full table

In [44]:
epath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LOD_calcs.csv"
full_table.to_csv(epath, index=False)